In [39]:
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
from pipeline.pupil import ConfigDeeplabcut,Tracking
import os
import cv2
import deeplabcut as dlc

In [11]:
from deeplabcut.utils import auxiliaryfunctions
from pipeline.utils import DLC_tools

In [81]:
key = {'animal_id': 26645, 'session': 2, 'scan_idx': 18}


In [64]:
# change config_path if we were to update DLC model configuration
temp_config = (ConfigDeeplabcut & dict(
    config_path='/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml')).fetch1()


In [65]:
# save config_path into the key
key['config_path'] = temp_config['config_path']

config = auxiliaryfunctions.read_config(temp_config['config_path'])
config['config_path'] = temp_config['config_path']
config['shuffle'] = temp_config['shuffle']
config['trainingsetindex'] = temp_config['trainingsetindex']

trainFraction = config['TrainingFraction'][config['trainingsetindex']]
DLCscorer = auxiliaryfunctions.GetScorerName(
    config, config['shuffle'], trainFraction)


In [66]:
# make needed directories
# tracking_dir, _ = self.create_tracking_directory(key)
vid_path = Tracking.Deeplabcut().get_video_path(key)
vid_dir = os.path.dirname(os.path.normpath(vid_path))
            
tracking_dir_name = os.path.basename(
                os.path.normpath(vid_path)).split('.')[0] + '_tracking'
tracking_dir = os.path.join(vid_dir, tracking_dir_name)


In [67]:
# make a short video (5 seconds long)
# short_video_path, original_width, original_height, mid_frame_num = DLC_tools.make_short_video(
#     tracking_dir)

case = os.path.basename(os.path.normpath(tracking_dir)).split('_tracking')[0]
input_video_path = os.path.join(tracking_dir, case + '.avi')
cap = cv2.VideoCapture(input_video_path)

original_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
original_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
mid_frame_num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)/2)

suffix = '_short.avi'
short_video_path = os.path.join(tracking_dir, 'short', case + suffix)



In [68]:
# add original width and height to config
config['original_width'] = original_width
config['original_height'] = original_height

# save info about short video
key['short_vid_starting_index'] = mid_frame_num

short_h5_path = short_video_path.split('.')[0] + DLCscorer + '.h5'


In [69]:
short_h5_path

'/mnt/scratch10/Two-Photon/Kayla/2021-12-14_14-28-04/26645_2_00018_beh_tracking/short/26645_2_00018_beh_shortDeepCut_resnet50_pupil_trackFeb12shuffle1_600000.h5'

In [ ]:
# predict using the short video
# DLC_tools.predict_labels(short_video_path, config)
# gputouse=0
# destfolder = os.path.dirname(short_video_path)
# dlc.analyze_videos(config=config['config_path'], videos=[short_video_path], videotype='avi', shuffle=config['shuffle'],
#                        trainingsetindex=config['trainingsetindex'], gputouse=gputouse, save_as_csv=False, destfolder=destfolder)


In [74]:
# obtain the cropping coordinates from the prediciton on short video
# this won't work after the fact because it removes this directory after insert
# cropped_coords = DLC_tools.obtain_cropping_coords(short_h5_path, DLCscorer, config)


In [78]:
from pipeline import pupil

In [80]:
key

{'animal_id': 26645,
 'session': 2,
 'scan_idx': 18,
 'config_path': '/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml',
 'short_vid_starting_index': 0}

In [79]:
pupil.FittedPupil.Circle & key

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,tracking_method method used for pupil tracking,frame_id frame id with matlab based 1 indexing,"center center of the circle in (x, y) of image",radius radius of the circle,visible_portion portion of visible pupil area given a fitted circle frame. Please refer DLC_tools.PupilFitting.detect_visible_pupil_area for more details
26645,2,18,2,0,=BLOB=,nan,-3.0
26645,2,18,2,1,=BLOB=,nan,-3.0
26645,2,18,2,2,=BLOB=,nan,-3.0
26645,2,18,2,3,=BLOB=,nan,-3.0
26645,2,18,2,4,=BLOB=,nan,-3.0
26645,2,18,2,5,=BLOB=,nan,-3.0
26645,2,18,2,6,=BLOB=,nan,-3.0
26645,2,18,2,7,=BLOB=,nan,-3.0
26645,2,18,2,8,=BLOB=,nan,-3.0
26645,2,18,2,9,=BLOB=,nan,-3.0


In [82]:
Tracking.Deeplabcut & key

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,tracking_method method used for pupil tracking,short_vid_starting_index middle frame index of the original video,cropped_x0 start width coord wrt original video,cropped_x1 end width coord wrt original video,cropped_y0 start height coord wrt original video,cropped_y1 end height coord wrt original video,added_pixels number of pixels added around the cropping coords,config_path path to deeplabcut config yaml
26645,2,18,2,74423,23,365,0,281,100,/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml


In [83]:
ConfigDeeplabcut & (Tracking.Deeplabcut & key)

config_path path to deeplabcut config,shuffle shuffle number used for the trained dlc model. Needed for dlc.analyze_videos,trainingsetindex trainingset index used for the trained dlc. model. Needed for dlc.analyze_videos
/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml,1,0


In [84]:
dlc_config = (ConfigDeeplabcut & (
                Tracking.Deeplabcut & key)).fetch1()

In [85]:
dlc_config

{'config_path': '/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml',
 'shuffle': 1,
 'trainingsetindex': 0}

In [86]:
config = auxiliaryfunctions.read_config(dlc_config['config_path'])
config['config_path'] = dlc_config['config_path']
config['shuffle'] = dlc_config['shuffle']
config['trainingsetindex'] = dlc_config['trainingsetindex']


In [90]:
avi_path = (pupil.Eye & key).get_video_path()
nframes = (pupil.Eye & key).fetch1('total_frames')

# find path to original video symlink
base_path = os.path.splitext(avi_path)[0] + '_tracking'
video_path = os.path.join(base_path, os.path.basename(avi_path))

config['orig_video_path'] = video_path


In [94]:
# find croppoing coords
cropped_coords = (Tracking.Deeplabcut & key).fetch1(
    'cropped_x0', 'cropped_x1', 'cropped_y0', 'cropped_y1')

config['cropped_coords'] = cropped_coords


In [ ]:

pupil_fit = DLC_tools.DeeplabcutPupilFitting(
    config=config, bodyparts='all', cropped=True)


In [ ]:
@schema
class Tracking(dj.Computed):
    definition = """
    -> Eye
    -> shared.TrackingMethod
    ---
    tracking_ts=CURRENT_TIMESTAMP   : timestamp  # automatic
    """

    class Deeplabcut(dj.Part):
        definition = """
        -> master
        ---
        short_vid_starting_index    : int unsigned          # middle frame index of the original video
        cropped_x0                  : smallint unsigned     # start width coord wrt original video
        cropped_x1                  : smallint unsigned     # end width coord wrt original video
        cropped_y0                  : smallint unsigned     # start height coord wrt original video
        cropped_y1                  : smallint unsigned     # end height coord wrt original video
        added_pixels                : smallint unsigned     # number of pixels added around the cropping coords
        config_path                 : varchar(128)          # path to deeplabcut config yaml        
        """

        def get_video_path(self, key):
            """
            Input:
                key: dictionary
                    A key that consists of animal_id, session, and scan_idx
            """
            video_info = (experiment.Session() *
                          experiment.Scan.EyeVideo() & key).fetch1()
            video_path = lab.Paths().get_local_path(
                "{behavior_path}/{filename}".format(**video_info))
            return video_path

        def create_tracking_directory(self, key):
            """
            this function creates the following directory structure:
            video_original_dir
                |
                |------ video_original
                |------ tracking_dir (create_tracking_directory)
                            |------- symlink to video_original (add_symlink)
                            |------- compressed_cropped_dir
                                        |------- cropped_video (generated by make_compressed_cropped_video function)
                                        |------- h5 file for cropped video (generated by deeplabcut)
                                        |------- pickle for cropped video (generated by deeplabcut)
                            |------- short_dir
                                        |------- short_video (generated by make_short_video function)
                                        |------- h5 file for short video(generated by deeplabcut)
                                        |------- pickle for short video (generated by deeplabcut)
            Input:
                key: dictionary
                    a dictionary that contains mouse id, session, and scan idx.
            Return:
                tracking_dir: string
                    a string that specifies the path to the tracking directory
            """

            print("Generating tracking directory for ", key)

            vid_path = self.get_video_path(key)
            vid_dir = os.path.dirname(os.path.normpath(vid_path))
            tracking_dir_name = os.path.basename(
                os.path.normpath(vid_path)).split('.')[0] + '_tracking'

            tracking_dir = os.path.join(vid_dir, tracking_dir_name)

            hardlink_path = os.path.join(
                tracking_dir, os.path.basename(os.path.normpath(vid_path)))

            if not os.path.exists(tracking_dir):

                os.mkdir(tracking_dir)
                os.mkdir(os.path.join(tracking_dir, 'compressed_cropped'))
                os.mkdir(os.path.join(tracking_dir, 'short'))

                os.link(vid_path, hardlink_path)

            else:
                print('{} already exists!'.format(tracking_dir))
                print('Removing existing tracking directory and recreating')

                shutil.rmtree(tracking_dir)

                os.mkdir(tracking_dir)
                os.mkdir(os.path.join(tracking_dir, 'compressed_cropped'))
                os.mkdir(os.path.join(tracking_dir, 'short'))

                os.link(vid_path, hardlink_path)

            return tracking_dir, hardlink_path

        def make(self, key):
            """
            Use Deeplabcut to label pupil and eyelids
            """

            print('Tracking labels with Deeplabcut!')

            # change config_path if we were to update DLC model configuration
            temp_config = (ConfigDeeplabcut & dict(
                config_path='/mnt/lab/DeepLabCut/pupil_track-Donnie-2019-02-12/config.yaml')).fetch1()

            # save config_path into the key
            key['config_path'] = temp_config['config_path']

            config = auxiliaryfunctions.read_config(temp_config['config_path'])
            config['config_path'] = temp_config['config_path']
            config['shuffle'] = temp_config['shuffle']
            config['trainingsetindex'] = temp_config['trainingsetindex']

            trainFraction = config['TrainingFraction'][config['trainingsetindex']]
            DLCscorer = auxiliaryfunctions.GetScorerName(
                config, config['shuffle'], trainFraction)

            # make needed directories
            tracking_dir, _ = self.create_tracking_directory(key)

            # make a short video (5 seconds long)
            short_video_path, original_width, original_height, mid_frame_num = DLC_tools.make_short_video(
                tracking_dir)

            # add original width and height to config
            config['original_width'] = original_width
            config['original_height'] = original_height

            # save info about short video
            key['short_vid_starting_index'] = mid_frame_num

            short_h5_path = short_video_path.split('.')[0] + DLCscorer + '.h5'

            # predict using the short video
            DLC_tools.predict_labels(short_video_path, config)

            # obtain the cropping coordinates from the prediciton on short video
            cropped_coords = DLC_tools.obtain_cropping_coords(
                short_h5_path, DLCscorer, config)

            # add 100 pixels around cropping coords. Ensure that it is within the original dim
            pixel_num = 100
            cropped_coords = DLC_tools.add_pixels(cropped_coords=cropped_coords,
                                                  original_width=original_width,
                                                  original_height=original_height,
                                                  pixel_num=pixel_num)

            # make a compressed and cropped video
            compressed_cropped_video_path = DLC_tools.make_compressed_cropped_video(
                tracking_dir, cropped_coords)

            # predict using the compressed and cropped video
            DLC_tools.predict_labels(compressed_cropped_video_path, config)

            key = dict(key, cropped_x0=cropped_coords['cropped_x0'],
                       cropped_x1=cropped_coords['cropped_x1'],
                       cropped_y0=cropped_coords['cropped_y0'],
                       cropped_y1=cropped_coords['cropped_y1'],
                       added_pixels=pixel_num)

            self.insert1(key)

            # delete short video directory
            shutil.rmtree(os.path.dirname(short_video_path))

            # delete compressed and cropped video
            os.remove(compressed_cropped_video_path)

    def make(self, key):
        print("Tracking for case {}".format(key))

        if key['tracking_method'] == 1:
            self.insert1(key)
            self.ManualTracking().make(key)
        elif key['tracking_method'] == 2:
            self.insert1(key)
            self.Deeplabcut().make(key)
        else:
            msg = 'Unrecognized Tracking method {}'.format(
                key['tracking_method'])
            raise PipelineException(msg)


In [ ]:

    def make(self, key):
        print("Fitting:", key)

        self.insert1(key)

        avi_path = (Eye & key).get_video_path()
        nframes = (Eye & key).fetch1('total_frames')

        data_circle = []
        data_ellipse = []

        # deeplabcut 2
        elif key['tracking_method'] == 2:

            dlc_config = (ConfigDeeplabcut & (
                Tracking.Deeplabcut & key)).fetch1()

            config = auxiliaryfunctions.read_config(dlc_config['config_path'])
            config['config_path'] = dlc_config['config_path']
            config['shuffle'] = dlc_config['shuffle']
            config['trainingsetindex'] = dlc_config['trainingsetindex']

            # find path to original video symlink
            base_path = os.path.splitext(avi_path)[0] + '_tracking'
            video_path = os.path.join(base_path, os.path.basename(avi_path))

            config['orig_video_path'] = video_path

            # find croppoing coords
            cropped_coords = (Tracking.Deeplabcut & key).fetch1(
                'cropped_x0', 'cropped_x1', 'cropped_y0', 'cropped_y1')

            config['cropped_coords'] = cropped_coords

            pupil_fit = DLC_tools.DeeplabcutPupilFitting(
                config=config, bodyparts='all', cropped=True)

            for bodypart in pupil_fit.bodyparts:
                self.EyePoints.insert1({**key,
                    'label': bodypart,
                    'x': pupil_fit.df_bodyparts[bodypart]['x'].values,
                    'y': pupil_fit.df_bodyparts[bodypart]['y'].values       
                })

            for frame_num in tqdm(range(nframes)):

                fit_dict = pupil_fit.fitted_core(frame_num=frame_num)

                # circle info
                center = fit_dict['circle_fit']['center']
                radius = fit_dict['circle_fit']['radius']
                visible_portion = fit_dict['circle_visible']['visible_portion']

                data_circle.append([center, radius, visible_portion])

                # ellipse info
                center = fit_dict['ellipse_fit']['center']
                major_radius = fit_dict['ellipse_fit']['major_radius']
                minor_radius = fit_dict['ellipse_fit']['minor_radius']
                rotation_angle = fit_dict['ellipse_fit']['rotation_angle']
                visible_portion = fit_dict['ellipse_visible']['visible_portion']

                data_ellipse.append([center, major_radius, minor_radius,
                                     rotation_angle, visible_portion])


        data_circle = np.array(data_circle)
        data_ellipse = np.array(data_ellipse)
        # now filter out the outliers by 5.5 std away from mean
        rejected_ind = DLC_tools.filter_by_fitting_std(
            data=data_circle, fitting_method='circle', std_magnitude=5.5)

        data_circle[rejected_ind] = None, None, -3.0

        common_entry = np.array(list(key.values()))
        common_matrix = np.tile(common_entry, (nframes, 1))

        data_circle = np.hstack(
            (common_matrix, np.arange(nframes).reshape(-1, 1), data_circle))

        # insert data
        with dj.config(enable_python_native_blobs = True):
            self.Circle.insert(data_circle)

        # now repeat the process for ellipse
        rejected_ind = DLC_tools.filter_by_fitting_std(
            data=data_ellipse, fitting_method='ellipse', std_magnitude=5.5)

        data_ellipse[rejected_ind, :] = None, None, None, None, -3.0

        data_ellipse = np.hstack(
            (common_matrix, np.arange(nframes).reshape(-1, 1), data_ellipse))

        with dj.config(enable_python_native_blobs = True):

            self.Ellipse.insert(data_ellipse)

In [ ]:

class DeeplabcutPupilFitting(DeeplabcutPlotBodyparts):
    def __init__(self, config, bodyparts='all', cropped=False, filtering=None):
        """
        Input:
            config: dictionary
                A dictionary that contains animal id, session, scan idx, and a path to config
            bodyparts: list
                A list that contains bodyparts to plot. Each bodypart is in a string format. If none provided,
                then by default it plots ALL existing bodyplots in config.yaml file.
        """
        super().__init__(config, bodyparts=bodyparts, cropped=cropped)

        self.complete_eyelid_graph = {'eyelid_top': 'eyelid_top_right',
                                      'eyelid_top_right': 'eyelid_right',
                                      'eyelid_right': 'eyelid_right_bottom',
                                      'eyelid_right_bottom': 'eyelid_bottom',
                                      'eyelid_bottom': 'eyelid_bottom_left',
                                      'eyelid_bottom_left': 'eyelid_left',
                                      'eyelid_left': 'eyelid_left_top',
                                      'eyelid_left_top': 'eyelid_top'}

        self._circle_threshold_num = 3
        self._ellipse_threshold_num = 6
        self._circle_color = (0, 255, 0)
        self._ellipse_color = (0, 0, 255)

    @property
    def circle_threshold_num(self):
        return self._circle_threshold_num

    @circle_threshold_num.setter
    def circle_threshold_num(self, value):
        if value > len(self.complete_eyelid_graph.keys()):
            raise ValueError("value must be equal to or less than {}!".format(
                len(self.complete_eyelid_graph.keys())))
        else:
            self._circle_threshold_num = value

    @property
    def ellipse_threshold_num(self):
        return self._ellipse_threshold_num

    @ellipse_threshold_num.setter
    def ellipse_threshold_num(self, value):
        if value > len(self.complete_eyelid_graph.keys()):
            raise ValueError("value must be equal to or less than {}!".format(
                len(self.complete_eyelid_graph.keys())))

        # 5 is the minimum number needed for ellipse fitting
        elif value < 5:
            raise ValueError("value must be equal to or more than 5!")
        else:
            self._ellipse_threshold_num = value

    @property
    def circle_color(self):
        return self._circle_color

    @circle_color.setter
    def circle_color(self, value):
        self._circle_color = value

    @property
    def ellipse_color(self):
        return self._ellipse_color

    @ellipse_color.setter
    def ellipse_color(self, value):
        self._ellipse_color = value

    def coords_pcutoff(self, frame_num):
        """
        Given a frame number, return bpindex, x & y coordinates that meet pcutoff criteria
        Input:
            frame_num: int
                A desired frame number
        Output:
            bpindex: list
                A list of integers that match with bodypart. For instance, if the bodypart is ['A','B','C']
                and only 'A' and 'C'qualifies the pcutoff, then bpindex = [0,2]
            x_coords: pandas series
                A pandas series that contains coordinates whose values meet pcutoff criteria
            y_coords: pandas series
                A pandas series that contains coordinates whose values meet pcutoff criteria
        """
        frame_num_tf = self.tf_likelihood_array[frame_num, :]
        bpindex = [i for i, x in enumerate(frame_num_tf) if x]

        df_x_coords = self.df_bodyparts_x.loc[frame_num, :][bpindex]
        df_y_coords = self.df_bodyparts_y.loc[frame_num, :][bpindex]

        return bpindex, df_x_coords, df_y_coords

    def connect_eyelids(self, frame_num, frame):
        """
        connect eyelid labels with a straight line. If a label is missing, do not connect and skip to the next label.
        Input:
            frame_num: int
                A desired frame number
            frame: numpy array
                A frame to be fitted
        Output:
            A dictionary containing the fitted frame and its corresponding binary mask.
            For each key in dictionary:
                frame:
                    A numpy array frame with eyelids connected
                mask:
                    A binary numpy array
        """
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)

        _, df_x_coords, df_y_coords = self.coords_pcutoff(frame_num)
        eyelid_labels = [label for label in list(
            df_x_coords.index.get_level_values(0)) if 'eyelid' in label]

        for eyelid in eyelid_labels:
            next_bp = self.complete_eyelid_graph[eyelid]

            if next_bp not in eyelid_labels:
                continue

            coord_0 = tuple(
                map(int, map(round, [df_x_coords[eyelid].values[0], df_y_coords[eyelid].values[0]])))
            coord_1 = tuple(
                map(int, map(round, [df_x_coords[next_bp].values[0], df_y_coords[next_bp].values[0]])))
            # opencv has some issues with dealing with np objects. Cast it manually again
            frame = cv2.line(
                np.array(frame), coord_0, coord_1, color=(255, 0, 0), thickness=self.line_thickness)
            mask = cv2.line(
                mask, coord_0, coord_1, color=(255), thickness=self.line_thickness)

        # fill out the mask with 1s OUTSIDE of the mask, then invert 0 and 1
        # for cv2.floodFill, need a mask that is 2 pixels bigger than the input image
        new_mask = np.zeros((mask.shape[0]+2, mask.shape[1]+2), dtype=np.uint8)
        cv2.floodFill(mask, new_mask, seedPoint=(0, 0), newVal=124)

        final_mask = np.logical_not(new_mask).astype(int)[1:-1, 1:-1]

        # ax.imshow(mask)
        return {'frame': frame,
                'mask': final_mask,
                'eyelid_labels_num': len(eyelid_labels)}

    def fit_circle_to_pupil(self, frame_num, frame):
        """
        Fit a circle to the pupil if it meets the circle_threshold value
        Input:
            frame_num: int
                A desired frame number
            frame: numpy array
                A frame to be fitted 3D
        Output: dictionary
            A dictionary with the fitted frame, center and radius of the fitted circle. If fitting did
            not occur, return the original frame with center and raidus as None.
            For each key in dictionary:
                frame: numpy array
                    a numpy array of the frame with pupil circle
                center: tuple
                    coordinates of the center of the fitted circle. In tuple format
                radius: float
                    radius of the fitted circle in int format
                mask: numpy array
                    a binary mask for the fitted circle area
                pupil_labels_num: int
                    number of pupil labels used for fitting
        """

        mask = np.zeros(frame.shape, dtype=np.uint8)

        _, df_x_coords, df_y_coords = self.coords_pcutoff(frame_num)

        pupil_labels = [label for label in list(
            df_x_coords.index.get_level_values(0)) if 'pupil' in label]

        if len(pupil_labels) < self.circle_threshold_num:
            # print('Frame number: {} has only 2 or less pupil label. Skip fitting!'.format(
            #     frame_num))
            center = None
            radius = None
            final_mask = mask[:, :, 0]

        else:
            pupil_x = df_x_coords.loc[pupil_labels].values
            pupil_y = df_y_coords.loc[pupil_labels].values

            pupil_coords = list(zip(pupil_x, pupil_y))

            x, y, radius = smallest_enclosing_circle_naive(pupil_coords)

            center = (x, y)

            # opencv has some issues with dealing with np objects. Cast it manually again
            frame = cv2.circle(img=np.array(frame), center=(int(round(x)), int(round(y))),
                               radius=int(round(radius)), color=self.circle_color, thickness=self.line_thickness)

            mask = cv2.circle(img=mask, center=(int(round(x)), int(round(y))),
                              radius=int(round(radius)), color=self.circle_color, thickness=self.line_thickness)

            # fill out the mask with 1s OUTSIDE of the mask, then invert 0 and 1
            # for cv2.floodFill, need a mask that is 2 pixels bigger than the input image
            new_mask = np.zeros(
                (mask.shape[0]+2, mask.shape[1]+2), dtype=np.uint8)
            cv2.floodFill(mask, new_mask, seedPoint=(0, 0), newVal=1)
            final_mask = np.logical_not(new_mask).astype(int)[1:-1, 1:-1]

        return {'frame': frame,
                'center': center,
                'radius': radius,
                'mask': final_mask,
                'pupil_labels_num': len(pupil_labels)}

    def fit_ellipse_to_pupil(self, frame_num, frame):
        """
        Fit an ellipse to pupil iff there exist more than 6 labels. If less than 6, return None
        Input:
            frame_num: int
                A desired frame number
            frame: numpy array
                A frame to be fitted in 3D
        Output: dictionary
            A dictionary with the fitted frame, center and radius of the fitted ellipse. If fitting did
            not occur, return None.
            For each key in dictionary:
                frame: numpy array
                    a numpy array of the frame with pupil ellipse
                center: tuple 
                    coordinates of the center of the fitted ellipse in tuple of floats
                mask: numpy array
                    a binary mask for the fitted ellipse area
                major_radius: float
                    major radius of the fitted ellipse
                minor_radius: float
                    minor radius of the fitted ellipse
                rotation_angle: float
                    angle from degree 0 to major_radius                
                pupil_labels_num: int
                    number of pupil labels used for fitting
        """

        mask = np.zeros(frame.shape, dtype=np.uint8)

        _, df_x_coords, df_y_coords = self.coords_pcutoff(frame_num)

        pupil_labels = [label for label in list(
            df_x_coords.index.get_level_values(0)) if 'pupil' in label]

        if len(pupil_labels) < self.ellipse_threshold_num:
            # print('Frame number: {} has only 2 or less pupil label. Skip fitting!'.format(
            #     frame_num))
            center = None
            major_radius = None
            minor_radius = None
            rotation_angle = None
            final_mask = mask[:, :, 0]

        else:
            pupil_x = df_x_coords.loc[pupil_labels].values.round().astype(
                np.int32)
            pupil_y = df_y_coords.loc[pupil_labels].values.round().astype(
                np.int32)

            pupil_coords = np.array(
                list(zip(pupil_x, pupil_y))).reshape((-1, 1, 2))

            # https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#fitellipse
            # Python: cv.FitEllipse2(points) → Box2D
            # https://docs.opencv.org/3.4.5/db/dd6/classcv_1_1RotatedRect.html
            rotated_rect = cv2.fitEllipse(pupil_coords)

            # https://docs.opencv.org/3.0-beta/modules/imgproc/doc/drawing_functions.html#ellipse
            # cv2.ellipse(img, box, color[, thickness[, lineType]]) → img
            frame = cv2.ellipse(np.array(
                frame), rotated_rect, color=self.ellipse_color, thickness=self.line_thickness)
            mask = cv2.ellipse(np.array(
                mask), rotated_rect, color=self.ellipse_color, thickness=self.line_thickness)

            # fill out the mask with 1s OUTSIDE of the mask, then invert 0 and 1
            # for cv2.floodFill, need a mask that is 2 pixels bigger than the input image
            new_mask = np.zeros(
                (mask.shape[0]+2, mask.shape[1]+2), dtype=np.uint8)
            cv2.floodFill(mask, new_mask, seedPoint=(0, 0), newVal=1)
            final_mask = np.logical_not(new_mask).astype(int)[1:-1, 1:-1]

            center = rotated_rect[0]
            major_radius = rotated_rect[1][1]/2.0
            minor_radius = rotated_rect[1][0]/2.0
            rotation_angle = rotated_rect[2]

        return {'frame': frame,
                'center': center,
                'mask': final_mask,
                'major_radius': major_radius,
                'minor_radius': minor_radius,
                'rotation_angle': rotation_angle,
                'pupil_labels_num': len(pupil_labels)}

    def detect_visible_pupil_area(self, eyelid_connect_dict, fit_dict, fitting_method=None):
        """
        Given a frame, find a visible part of the pupil by finding the intersection of pupil and eyelid masks
        If pupil mask does not exist(i.e. label < 3), return None
        Input:
            eyelid_connect_dict: dicionary
                An output dictionary from method 'connect_eyelids'
            fit_dict: dictionary
                An output dictionary from either of the method 'fit_circle_to_pupil' or 'fit_ellipse_to_pupil'
            fitting_method: string
                A string indicates whether the fitted method was an 'ellipse' or a 'circle'. 
        Output:
            A dictionary that contains the following:
                mask: numpy array
                    A 3D mask that depicts visible area of pupil. 
                    If no visible area provided, then it is an np.zeros
                visible_portion: signed float
                    if visible area exists, then value ranges from 0.0 to 1.0
                    if equal to -1.0, not all eyelid labels exist, hence cannot find the visible area
                    if equal to -2.0, number of pupil labels do not meet the threshold of the fitting method, hence no fitting performed
                    if equal to -3.0, not all eyelid labels exist AND not enough pupil labels to meet the threshold of the fitting method
        """

        color_mask = np.zeros(
            shape=[*eyelid_connect_dict['mask'].shape, 3], dtype=np.uint8)

        if fitting_method == 'circle':
            threshold = self.circle_threshold_num

        elif fitting_method == 'ellipse':
            threshold = self.ellipse_threshold_num

        else:
            raise ValueError(
                'fitting_method must be provided! It must be either a "circle" or an "ellipse"')

        if fit_dict['pupil_labels_num'] >= threshold and eyelid_connect_dict['eyelid_labels_num'] == 8:

            visible_mask = np.logical_and(
                fit_dict['mask'], eyelid_connect_dict['mask']).astype(int)

            # 126,0,255 for the color
            color_mask[visible_mask == 1, 0] = 126
            color_mask[visible_mask == 1, 2] = 255

            visible_portion = visible_mask.sum() / fit_dict['mask'].sum()

        elif fit_dict['pupil_labels_num'] >= threshold and eyelid_connect_dict['eyelid_labels_num'] != 8:
            visible_portion = -1.0

        elif fit_dict['pupil_labels_num'] < threshold and eyelid_connect_dict['eyelid_labels_num'] == 8:
            visible_portion = -2.0

        elif fit_dict['pupil_labels_num'] < threshold and eyelid_connect_dict['eyelid_labels_num'] != 8:
            visible_portion = -3.0

        return dict(mask=color_mask, visible_portion=visible_portion)

    def fitted_core(self, frame_num):
        """
        Input:
            fig: figure object
                Figure object created by configure_plot method
            ax: axis object
                Axis object created by configure_plot method
            frame_num: int
                frame number of the video to be analyzed
            fitting_method: string
                It must be either 'circle' or 'ellipse'. If not provided, raising an error
        Output:
            A dictionary
                'ax_frame': A fitted axis object
                'ax_scatter': A scatter axis object that shows where labels are
                'ax_mask':
        """
        # it's given in 3 channels but every channel is the same i.e. grayscale

        image = self.clip._read_specific_frame(frame_num)

        if self.cropped:

            x1 = self.cropped_coords[0]
            x2 = self.cropped_coords[1]
            y1 = self.cropped_coords[2]
            y2 = self.cropped_coords[3]

            image = image[y1:y2, x1:x2]

        eyelid_connected = self.connect_eyelids(
            frame_num=frame_num, frame=image)

        circle_fit = self.fit_circle_to_pupil(
            frame_num=frame_num, frame=eyelid_connected['frame'])

        ellipse_fit = self.fit_ellipse_to_pupil(
            frame_num=frame_num, frame=eyelid_connected['frame'])

        circle_visible = self.detect_visible_pupil_area(eyelid_connect_dict=eyelid_connected,
                                                        fit_dict=circle_fit,
                                                        fitting_method='circle')

        ellipse_visible = self.detect_visible_pupil_area(eyelid_connect_dict=eyelid_connected,
                                                         fit_dict=ellipse_fit,
                                                         fitting_method='ellipse')

        return {'circle_fit': circle_fit,
                'ellipse_fit': ellipse_fit,
                'circle_visible': circle_visible,
                'ellipse_visible': ellipse_visible}

    def plot_fitted_frame(self, frame_num, ax=None, fitting_method='circle', save_fig=False):

        if ax is None:
            fig, ax = self.configure_plot()

        # plot bodyparts above the pcutoff
        bpindex, x_coords, y_coords = self.coords_pcutoff(frame_num)
        ax_scatter = ax.scatter(x_coords.values, y_coords.values, s=self.dotsize**2,
                                color=self._label_colors(bpindex), alpha=self.alphavalue)

        fitted_core_dict = self.fitted_core(frame_num)

        if fitting_method == 'circle':

            circle_frame = ax.imshow(fitted_core_dict['circle_fit']['frame'])
            circle_mask = ax.imshow(
                fitted_core_dict['circle_visible']['mask'], alpha=0.2)

        elif fitting_method == 'ellipse':

            ellipse_frame = ax.imshow(fitted_core_dict['ellipse_fit']['frame'])
            ellipse_mask = ax.imshow(
                fitted_core_dict['ellipse_visible']['mask'], alpha=0.2)

        else:
            raise ValueError(
                'fitting method must be either an ellipse or a circle!')

        ax.set_title('frame num: ' + str(frame_num), fontsize=self.fontsize)

        ax.axis('off')
        plt.tight_layout()

        if save_fig:
            plt.savefig(os.path.join(
                self.compressed_cropped_dir_path, 'fitted_frame_' + str(frame_num) + '.png'))

        if ax is not None:
            return ax

    def plot_fitted_multi_frames(self, start, end, fitting_method='circle', save_gif=False):

        fig, ax = self.configure_plot()

        plt_list = []

        for frame_num in range(start, end):

            self.plot_fitted_frame(frame_num=frame_num,
                                   ax=ax, fitting_method=fitting_method)

            fig.canvas.draw()

            data = np.fromstring(fig.canvas.tostring_rgb(),
                                 dtype=np.uint8, sep='')
            data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
            plt_list.append(data)

            display.clear_output(wait=True)
            display.display(pl.gcf())
            time.sleep(0.5)

            plt.cla()

        if save_gif:
            gif_path = self.video_path.split('.')[0] + '_fitted_' + \
                str(start) + '_' + str(end) + '.gif'
            imageio.mimsave(gif_path, plt_list, fps=1)

        plt.close('all')

    def make_movie(self, start, end, fitting_method='circle'):

        import matplotlib.animation as animation

        # initlize with start frame
        fig, ax = self.configure_plot()

        self.plot_fitted_frame(frame_num=start, ax=ax,
                               fitting_method=fitting_method)

        def update_frame(frame_num):

            # clear out the axis
            plt.cla()

            self.plot_fitted_frame(frame_num=frame_num,
                                   ax=ax, fitting_method=fitting_method)

        ani = animation.FuncAnimation(fig, update_frame, range(
            start+1, end))  # , interval=int(1/self.clip.FPS)
        # ani = animation.FuncAnimation(fig, self.plot_fitted_frame, 10)
        writer = animation.writers['ffmpeg'](fps=self.clip.FPS)

        if self.cropped:
            crop_flag = 'cropped'
        else:
            crop_flag = 'orig'
        save_video_path = os.path.join(self.base_dir,
                                       '{}_{}_{}_labeled.avi'.format(crop_flag, start, end))

        ani.save(save_video_path, writer=writer, dpi=self.dpi)

        return ani


In [ ]:

class DeeplabcutPlotBodyparts():

    def __init__(self, config, bodyparts='all', cropped=False):
        """
        Input:
            config: dictionary
                A dictionary that contains animal id, session, scan idx, and a path to config
            bodyparts: list
                A list that contains bodyparts to plot. Each bodypart is in a string format. If none provided,
                then by default it plots ALL existing bodyplots in config.yaml file.
            cropped: boolean
                whether to crop the video or not. Default False
            filtering (dict):
        """

        self.config = config

        if isinstance(bodyparts, list):
            self.bodyparts = bodyparts
        else:
            self.bodyparts = self.config['bodyparts']

        self.cropped = cropped
        self.cropped_coords = config['cropped_coords']

        self.shuffle = self.config['shuffle']
        self.trainingsetindex = self.config['trainingsetindex']

        self.project_path = self.config['project_path']
        self.orig_video_path = self.config['orig_video_path']
        self.base_dir = os.path.dirname(self.orig_video_path)
        self.compressed_cropped_dir_path = os.path.join(
            os.path.dirname(self.orig_video_path), 'compressed_cropped')
        self._case = os.path.basename(config['orig_video_path']).split('.')[0]
        self.clip = video_processor.VideoProcessorCV(
            fname=self.orig_video_path)

        self._trainFraction = self.config['TrainingFraction'][self.trainingsetindex]
        self._DLCscorer = auxiliaryfunctions.GetScorerName(
            self.config, self.shuffle, self._trainFraction)

        self.label_path = os.path.join(
            self.compressed_cropped_dir_path, self._case + '_compressed_cropped' + self._DLCscorer + '.h5')

        self.df_label = pd.read_hdf(self.label_path)

        self.df_bodyparts = self.df_label[self._DLCscorer][self.bodyparts]

        self.df_bodyparts_likelihood = self.df_bodyparts.iloc[:, self.df_bodyparts.columns.get_level_values(
            1) == 'likelihood']

        self.df_bodyparts_x = self.df_bodyparts.iloc[:,
                                                     self.df_bodyparts.columns.get_level_values(1) == 'x']
        self.df_bodyparts_y = self.df_bodyparts.iloc[:,
                                                     self.df_bodyparts.columns.get_level_values(1) == 'y']

        # in mm. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3310398/#R13
        self._pupil_diameter = 3.0 

        # obtain median left to right eyelid distance
        right = self.df_bodyparts['eyelid_right'].values[:, :2]
        left = self.df_bodyparts['eyelid_left'].values[:, :2]

        self.median_left_right = np.median(
            np.sqrt(np.einsum('ij,ij->i', left-right, left-right)))

        # obtain pixel to diameter ratio
        self._pixel_to_diameter_ratio = self.median_left_right/self._pupil_diameter

        if not self.cropped:

            self.nx = self.clip.width()
            self.ny = self.clip.height()
            self.df_bodyparts_x = self.df_bodyparts.iloc[:,
                                                         self.df_bodyparts.columns.get_level_values(1) == 'x'] + self.cropped_coords[0]
            self.df_bodyparts_y = self.df_bodyparts.iloc[:,
                                                         self.df_bodyparts.columns.get_level_values(1) == 'y'] + self.cropped_coords[2]

        else:

            if self.cropped_coords is None:
                raise ValueError(
                    "cropped_coords are not provided! Must be in list with 4 elmnts long!")

            if len(self.cropped_coords) != 4:
                raise ValueError(
                    "Only provided {} coordinates! U need 4!".format(len(self.cropped_coords)))

            # self.df_bodyparts_x = self.df_bodyparts.iloc[:,
            #                                             self.df_bodyparts.columns.get_level_values(1) == 'x'] - self.cropped_coords[0]
            # self.df_bodyparts_y = self.df_bodyparts.iloc[:,
            #                                             self.df_bodyparts.columns.get_level_values(1) == 'y'] - self.cropped_coords[2]

            self.nx = self.clip.width() - self.cropped_coords[0]
            self.ny = self.clip.height() - self.cropped_coords[2]

        # plotting properties
        self._dotsize = 7
        self._line_thickness = 1
        self._pcutoff = self.config['pcutoff']
        self._colormap = self.config['colormap']
        self._label_colors = plotting.get_cmap(
            len(self.bodyparts), name=self._colormap)
        self._alphavalue = self.config['alphavalue']
        self._fig_size = [12, 8]
        self._dpi = 100
        self._fontsize = 30

        self.tf_likelihood_array = self.df_bodyparts_likelihood.values > self._pcutoff


In [ ]:
def smallest_enclosing_circle_naive(points):
    # Degenerate cases
    if len(points) == 0:
        return None
    elif len(points) == 1:
        return (points[0][0], points[0][1], 0)

    result = None
    # Try all unique triples
    for i in range(len(points)):
        p = points[i]
        for j in range(i + 1, len(points)):
            q = points[j]
            for k in range(j + 1, len(points)):
                r = points[k]
                c = make_circumcircle(p, q, r)
                if c is not None and (result is None or c[2] < result[2]) and \
                        all(is_in_circle(c, s) for s in points):
                    result = c

    if result is None:
        raise AssertionError()
    return result


In [ ]:

def make_circumcircle(a, b, c):
    # Mathematical algorithm from Wikipedia: Circumscribed circle
    ox = (min(a[0], b[0], c[0]) + max(a[0], b[0], c[0])) / 2.0
    oy = (min(a[1], b[1], c[1]) + max(a[1], b[1], c[1])) / 2.0
    ax = a[0] - ox
    ay = a[1] - oy
    bx = b[0] - ox
    by = b[1] - oy
    cx = c[0] - ox
    cy = c[1] - oy
    d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
    if d == 0.0:
        return None
    x = ox + ((ax*ax + ay*ay) * (by - cy) + (bx*bx + by*by)
              * (cy - ay) + (cx*cx + cy*cy) * (ay - by)) / d
    y = oy + ((ax*ax + ay*ay) * (cx - bx) + (bx*bx + by*by)
              * (ax - cx) + (cx*cx + cy*cy) * (bx - ax)) / d
    ra = math.hypot(x - a[0], y - a[1])
    rb = math.hypot(x - b[0], y - b[1])
    rc = math.hypot(x - c[0], y - c[1])
    return (x, y, max(ra, rb, rc))


In [ ]:
def is_in_circle(c, p):
    return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) <= c[2] * _MULTIPLICATIVE_EPSILON
